In [1]:
import pandas as pd

import library.IO as myio
import library.motion as mt
import library.comparison as cp
from library.constants import *
import matplotlib
matplotlib.use('Qt5Agg')

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
file_name_0 = "P01_B_cut"
marks_motion_0 = [7600, 7850]
label_motion_0 = "label_0"

file_name_1 = "P02_B_cut"
#marks_motion_1 = [7600, 8100]
marks_motion_1 = [7600, 7850]
label_motion_1 = "label_1"

weights_0 = {}
for joint in SIMPLIFIED_JOINTS:
    weights_0[joint] = 1
weights_0["Right Hand"] = 0
weights_0["Right Upper Arm"] = 0
weights_0["Right Forearm"] = 0

# if you need more than one group of weight, just use the code below and add it to weights_groups
# weights_1 = {}
# for joint in SIMPLIFIED_JOINTS:
#     weights_1[joint] = 1

weights_groups = pd.DataFrame([weights_0])

flag_output_xlsx = False  # output xlsx or not
output_types = ["Score", "Segment Angular Velocity"]
xlsx_filename = "example"

flag_show_animation = True  # generate the animation and play it
flag_output_gif = False  # generate the animation and output as gif
gif_file_name = "example"
flag_visualized_vector = False  # animation with visualized velocity or not
flag_heatmap = True  # animation with heatmap or not
flag_repeat = True  # animation will always repeat or not
visualized_vector = "Segment Angular Velocity"
heatmap_recording = "Score"

In [3]:
meta_data_0 = mt.MetaData(file_name_0, marks_motion_0[0], marks_motion_0[-1], label_motion_0)
meta_data_1 = mt.MetaData(file_name_1, marks_motion_1[0], marks_motion_1[-1], label_motion_1)

if flag_output_xlsx:
    xlsx_settings = myio.XlsxSetting(
        xlsx_filename=xlsx_filename,
        output_types=output_types
    )
else:
    xlsx_settings = None

if flag_output_gif or flag_show_animation:
    animation_settings = myio.AnimationSetting(
        flag_visualized_vector=flag_visualized_vector,
        flag_heatmap=flag_heatmap,
        flag_repeat=flag_repeat,
        visualized_vector=visualized_vector,
        heatmap_recording=heatmap_recording
    )
else:
    animation_settings = None

io = myio.MyIO(
    flag_output_xlsx=flag_output_xlsx,
    flag_show_animation=flag_show_animation,
    flag_output_gif=flag_output_gif,
    xlsx_settings=xlsx_settings,
    animation_settings=animation_settings,
    motions_meta=[meta_data_0, meta_data_1]
)

In [4]:
motions = io.get_motions()
comparison = cp.Comparison(weights_groups, marks_motion_0)

generating label_0 from P01_B_cut
generating label_1 from P02_B_cut


In [5]:
# for motion in motions:
#     motion.centre().confront()
#
# motions[1].synchronized_by(motions[0])
result = comparison.compare(motions[0], motions[1], io.get_comparison_types())


calculating the distances and score


In [7]:
io.output(motions,result)

generating the animation
